In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
df = pd.read_csv('mock_transactions.csv')
df1 = pd.read_csv('Merged_card_user.csv')

df1

In [ ]:
df['Errors?'] = df['Errors?'].fillna('None')

# 2. Remove backticks from 'Card Number' and 'CVV'
df['Card Number'] = df['Card Number'].str.replace('`', '', regex=False).astype('str')
df['CVV'] = df['CVV'].str.replace('`', '', regex=False).astype('str')

df1['Card Number'] = df1['Card Number'].str.replace('`', '', regex=False).astype('str')
df1['CVV'] = df1['CVV'].str.replace('`', '', regex=False).astype('str')
# 3. Remove '$' from 'Amount' and convert to float
df['Amount'] = df['Amount'].str.replace(',', '', regex=False)
# df1['Amount'] = df['Amount'].str.replace(',', '', regex=False)

# First, strip out the '$', then convert the resulting string to float
df['Amount'] = df['Amount'].str.replace('$', '', regex=False).astype(float)
# df1['Amount'] = df['Amount'].str.replace('$', '', regex=False).astype(float)

# Now df is updated accordingly
# You can verify by checking the head of the DataFrame
print(df.head())
# Assuming your DataFrame is named df and is fully cleaned
print(df1.head())



In [ ]:
df.to_json('mock_transactions.json', orient='records')
df1.to_json('Merged_card_user.json', orient='records')

In [ ]:
# print(df.head())
# Assuming your DataFrame is named df and is fully cleaned
df1.columns

In [ ]:
import pandas as pd
import json

# Load the data
merged_users_df = pd.read_json('Merged_card_user.json')
transactions_df = pd.read_json('mock_transactions.json')

# Ensure 'Card Number' is a string for consistent handling
merged_users_df['Card Number'] = merged_users_df['Card Number'].astype(str)
transactions_df['Card Number'] = transactions_df['Card Number'].astype(str)

# Group transactions by 'Card Number'
transactions_grouped = transactions_df.groupby('Card Number').apply(lambda x: x.to_dict(orient='records')).to_dict()

# Initialize the result dictionary
result_dict = {}

# Iterate over the user DataFrame
for _, user_row in merged_users_df.iterrows():
    user_id = user_row['User']
    card_number = user_row['Card Number']

    # If the user is not already in the dictionary, add their personal information
    if user_id not in result_dict:
        result_dict[user_id] = {
            "User": user_row['User'],
            "Person": user_row['Person'],
            "Current Age": user_row['Current Age'],
            "Retirement Age": user_row['Retirement Age'],
            "Birth Year": user_row['Birth Year'],
            "Birth Month": user_row['Birth Month'],
            "Gender": user_row['Gender'],
            "Address": user_row['Address'],
            "City": user_row['City'],
            "State": user_row['State'],
            "Zipcode": user_row['Zipcode'],
            "Latitude": user_row['Latitude'],
            "Longitude": user_row['Longitude'],
            "Per Capita Income - Zipcode": user_row['Per Capita Income - Zipcode'],
            "Yearly Income - Person": user_row['Yearly Income - Person'],
            "Total Debt": user_row['Total Debt'],
            "FICO Score": user_row['FICO Score'],
            "Num Credit Cards": user_row['Num Credit Cards'],
            "Cards": {}
        }

    # Add the card details and transactions under the "Cards" key
    result_dict[user_id]['Cards'][card_number] = {
        "Card Details": {
            "Card Type": user_row['Card Type'],
            "Expires": user_row['Expires'],
            "CVV": user_row['CVV'],
            "Has Chip": user_row['Has Chip'],
            "Credit Limit": user_row['Credit Limit'],
            "Account Balance": user_row['Account Balance'],
            "Acct Open Date": user_row['Acct Open Date'],
            "Year PIN Last Changed": user_row['Year PIN last Changed'],
            "Card on Dark Web": user_row['Card on Dark Web']
        },
        "Transactions": transactions_grouped.get(card_number, [])
    }

# Convert the result dictionary to JSON
json_output = json.dumps(result_dict, indent=4, default=str)

# Save the JSON output to a file
with open('enhanced_combined_data.json', 'w') as json_file:
    json_file.write(json_output)

print("JSON file saved successfully.")
